# Daily Results

## Initialisation

Basic approach to determine the project directory

In [1]:
import os
import sys
import glob

from datetime import datetime
import time

import csv
import json

In [2]:
projdir = os.path.realpath(os.path.join(sys.path[0], '..'))

In [3]:
EVENTS_DIR = 'events'
CONFIG_DIR = 'config'
SESSIONS_DIR = 'sessions'
GPSDATA_DIR = 'gpsdata'

In [4]:
CONFIG_JSON = 'config.json'
ENTRANTS_CSV = 'entrants.csv'
SESSIONS_JSON = 'sessions.json'

In [5]:
DEFAULT_NATIONALITY = 'England'

In [6]:
try:
    wsw_full_refresh = int(os.environ['WSW_FULL_REFRESH'])
except:
    wsw_full_refresh = 1
    
verbose = True

## Generic Class

Generic class to ensure that all custom classes are printable

In [7]:
class Printable:
    def __repr__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

    def __str__(self):
        return str(self.__class__) + ": " + str(self.__dict__)
    
    def logInfo(self, msg):
        print('INFO:', msg)

    def logWarning(self, msg):
        print('WARNING:', msg)

    def logError(self, msg):
        print('ERROR:', msg)

## SpeedRun Class

Class to manage runs by an entrant

In [8]:
class SpeedRun(Printable):
    def __init__(self, courseId, startTime, duration, speed, cog):
        '''Initialise speed run object'''    
        
        self.courseId, self.startTime, self.duration, self.speed, self.cog = courseId, startTime, duration, speed, cog

## Entrant Class

Class to manage entrants

In [9]:
class Entrant(Printable):
    def __init__(self, headers, values):
        '''Initialise entrant object'''

        # Quickly check the number of values matches the header
        if len(headers) != len(values):
            raise ValueError('Unexpected number of fields for entrant ID {}'.format(values[0]))
        
        # Patch legacy headers to use current naming
        self.patchHeaders(headers)

        # Store entrant details in a dictionary - easy for applying report filters
        self.entrantDict = {}
        for i in range(len(values)):
            self.entrantDict[headers[i]] = values[i]
        self.entrantId = self.entrantDict['ID']
        
        # Patch and finally validate the resultant dictionary
        self.patchDictionary()
        self.validateDictionary()
        
        # Record GPS details
        self.recordGt31Details()

        # Note: A session is a complete day in WSW terms but may comprise of multiple courses
        self.sessions = {}


    def patchHeaders(self, headers):
        '''Tweak headers for consistency across years'''
        
        for i in range(len(headers)):
            # All years
            if headers[i] == 'Male/Female':
                headers[i] = 'Gender'

            # 2010
            elif headers[i] == 'Novice(Y/N)':
                headers[i] = 'First Timer'

            # 2010 + 2011
            elif headers[i] == 'requiredEntryType':
                headers[i] = 'EntryType'

            # 2010
            elif headers[i] == 'NMA Member':
                headers[i] = 'ISWC Member'
        

    def patchDictionary(self):
        '''Patch dictionary values to handle previous years'''

        # If nationality is not present then default to the country of residence
        if 'Nationality' not in self.entrantDict:
            self.entrantDict['Nationality'] = self.entrantDict['Country']
        
        # Convert England to United Kingdom so that ISO code + flag can be shown in results
        if self.entrantDict['Nationality'] in ('England', 'Scotland', 'Wales', 'Northern Ireland'):
            self.entrantDict['Nationality'] = 'United Kingdom'

        # 2011 had a status of "Novice"
        if self.entrantDict['Status'] == 'Novice':
            self.entrantDict['Status'] = 'Amateur'
            self.entrantDict['First Timer'] = 'Y'
            
        if 'First Timer' not in self.entrantDict:
            self.entrantDict['First Timer'] = '?'

        # 2016 had a status of "Youth"
        if self.entrantDict['Status'] == 'Youth':
            self.entrantDict['Status'] = 'Amateur'
            self.entrantDict['EntryType'] = 'Youth Weekend'

        # Simple patches
        for key in self.entrantDict:
            if key == 'Status':
                # 2010 + 2011
                if self.entrantDict[key] == 'Professional':
                    self.entrantDict[key] = 'Pro-Fleet'
                elif self.entrantDict[key] == 'Gold Fleet':
                    self.entrantDict[key] = 'Gold-Fleet'

                # 2012
                elif self.entrantDict[key] == 'Please select':
                    self.entrantDict[key] = 'Unknown'

            elif key == 'EntryType':
                # 2010 + 2011 + 2012 + 2013
                if self.entrantDict[key] in ('2 Days Only', '2 Days', '2-day'):
                    self.entrantDict[key] = '2-Day'

                # 2011 + 2012
                elif self.entrantDict[key] in ('1 Day Only', '1 Day', '1-day'):
                    self.entrantDict[key] = '1-Day'

                # 2010 + 2011 + 2012
                elif self.entrantDict[key] == '':
                    self.entrantDict[key] = 'Unknown'

            elif key in ('First Timer', 'UKWA Member', 'ISWC Member'):
                # Patching of Y/N flags can be applied to any year
                self.entrantDict[key] = self.entrantDict[key][:1].upper()
                if self.entrantDict[key] == '':
                    self.entrantDict[key] = '?'

        # ISWC may not be present for some years, simply default to '?'
        for key in ['ISWC Member']:
            if key not in self.entrantDict:
                self.entrantDict[key] = '?'


    def validateDictionary(self):
        '''Validate the dictionary by comparing to known values in the application configuration'''

        for key in config['Entrants']:
            if key in self.entrantDict:
                if self.entrantDict[key] not in config['Entrants'][key]:
                    raise ValueError('Unexpected "{}" for entrant ID {} - "{}"'.format(key, self.entrantId, self.entrantDict[key]))
            else:
                print(self.entrantDict)
                raise ValueError('Missing "{}" for entrant ID {}'.format(key, self.entrantId))


    def recordGt31Details(self):
        '''Record GT-31 details'''

        self.gt31SerialNumbers = set()
        if 'GT31 SN' in self.entrantDict:
            for gt31SerialNumber in str(self.entrantDict['GT31 SN'].replace(';', ',')).split(','):
                if gt31SerialNumber != '':
                    self.gt31SerialNumbers.add(gt31SerialNumber)


    def storeRun(self, courseDate, courseId, startTime, duration, speed, cog):
        '''Store run in memory'''
        
        speedRun = SpeedRun(courseId, startTime, duration, speed, cog)

        if courseDate not in self.sessions:
            self.sessions[courseDate] = []
            
        self.sessions[courseDate].append(speedRun)

## Course Class

Class to manage courses - start / end times

In [10]:
class Course(Printable):
    def __init__(self, entrants, courseDate, courseId, courseDict):
        '''Initialise course object'''

        self.entrants = entrants
        self.participants = {}

        self.courseDate = courseDate
        self.courseId = courseId
        self.startTime = courseDict['Start Time']
        self.endTime = courseDict['End Time']
        
        self.minStartTime = '23:59:59'
        self.maxStartTime = '00:00:00'
        self.numRuns = 0
        self.numGt31s = 0


    def getParticipant(self, gt31Ids, gt31Id, gt31Serial):
        '''Get participant from the GT-31 ID and serial'''

        if gt31Id not in gt31Ids:
            self.logWarning('Unrecognised GT-31 ID on {} - {}'.format(self.courseDate, gt31Id))

            entrantId = max(self.entrants.keys()) + 1
            entrantDict = {
                "Gender": "?",
                "First Name": "GT-31",
                "Family Name": gt31Id,
                "First Timer": "?",
                "Status": "Unknown",
                "Craft Type": "Unknown",
                "GT31 ID": gt31Id,
                "GT31 SN": gt31Serial,
                "EntryType": "Unknown",
                "UKWA Member": "?",
            }
            participant = Entrant(entrantId, entrantDict)

            self.entrants[entrantId] = participant
            self.participants[entrantId] = participant
            # TODO gt31Id
        else:
            if gt31Serial not in gt31Ids[gt31Id].gt31SerialNumbers:
                self.logWarning('Unrecognised GT-31 SN for {} {} ({}) on {} - {} vs {}'.format(
                        gt31Ids[gt31Id].firstName, gt31Ids[gt31Id].familyName, gt31Ids[gt31Id].craftType,
                        self.courseDate,
                        gt31Serial, gt31Ids[gt31Id].gt31SerialNumbers))

            participant = gt31Ids[gt31Id]
            self.participants[participant.entrantId] = participant
            
        return participant


    def storeRun(self, participant, startTime, duration, speed, cog):
        '''Store run in memory'''

        if startTime < self.minStartTime:
            self.minStartTime = startTime
        if startTime > self.maxStartTime:
            self.maxStartTime = startTime

        if startTime >= self.startTime and startTime <= self.endTime:
            participant.storeRun(self.courseDate, self.courseId, startTime, duration, speed, cog)

        self.numRuns += 1


    def loadRuns(self, csvPath, gt31Ids):
        '''Read speeds from CSV files'''

        prevGt31Id = None

        with open(csvPath, 'r') as f:
            csvReader = csv.reader(f)
            for values in csvReader:
                # GPSResults does not output a header line
                run, filename, startTime, duration, speed, cog = values

                # Split up record and format the GT-31 details
                gt31Id, gt31Serial, fileDate, fileTime = os.path.splitext(filename)[0].split('_')[:4]
                gt31Id = gt31Id.upper()

                # Validate each newly encountered GT-31
                if gt31Id != prevGt31Id:
                    participant = self.getParticipant(gt31Ids, gt31Id, gt31Serial)

                # Only store runs for recognised entrants (via GT-31 ID)
                if participant:
                    self.storeRun(participant, startTime, duration, speed, cog)
                
                prevGt31Id = gt31Id

            self.logInfo('{} runs by {} recognised participants down course {} on {} - {} to {}'.format(
                    self.numRuns, len(self.participants), self.courseId, self.courseDate, self.minStartTime, self.maxStartTime))
            
        if self.minStartTime < self.startTime:
            self.logWarning('Runs found before course {} opened on {} - earliest was {}'.format(
                self.courseId, self.courseDate, self.minStartTime))

        if self.maxStartTime > self.endTime:
            self.logWarning('Runs found after course {} closed on {} - latest was {}'.format(
                self.courseId, self.courseDate, self.maxStartTime))

## Session Class

Class to manage sessions

In [11]:
class Session(Printable):
    def __init__(self, entrants, dataPath, configPath):
        '''Initialise session object'''

        self.entrants = entrants
        self.dataPath = dataPath
        self.configPath = configPath

        self.date = os.path.basename(configPath)
        self.year = self.date[:4]

        self.courses = {}


    def loadConfig(self):
        '''Read config from JSON'''

        filename = os.path.join(self.configPath, SESSIONS_JSON)
        with open(filename, 'r', encoding='utf-8') as f:
            jsonTxt = f.read()
            courseDict = json.loads(jsonTxt)
            
            for courseId in courseDict:
                # Course ID should be uppercase but just to be sure...
                courseId = courseId.upper()
                if courseId in self.courses:
                    raise ValueError('Duplicate course "{}" for {}'.format(courseId, self.date))
                course = Course(self.entrants, self.date, courseId, courseDict[courseId])
                self.courses[courseId] = course


    def loadRuns(self, gt31Ids):
        '''Read speeds from CSV files'''

        csvPaths = sorted(glob.glob(os.path.join(self.dataPath, '*')))
        
        for csvPath in csvPaths:
            prefix, sessionDate, courseId = os.path.splitext(os.path.basename(csvPath))[0].split('_')
            suffix = os.path.splitext(csvPath)[1]

            # Verify the filename
            if suffix.lower() != '.csv':
                raise ValueError('Invalid file suffix "{}" for {}'.format(suffix, os.path.basename(csvPath)))
            if prefix.upper() != 'GPSDATA':
                raise ValueError('Invalid file prefix "{}" for {}'.format(prefix, os.path.basename(csvPath)))
            if sessionDate != self.date:
                raise ValueError('Invalid file date "{}" for {}'.format(sessionDate, os.path.basename(csvPath)))
            if courseId.upper() not in self.courses:
                raise ValueError('Invalid course "{}" for {}'.format(courseId, os.path.basename(csvPath)))
            
            course = self.courses[courseId.upper()]
            course.loadRuns(csvPath, gt31Ids)

## Event Class

Class to manage events

In [12]:
class Event():
    def __init__(self, path):
        self.path = path
        self.year = int(os.path.basename(path))

        self.entrants = {}
        self.gt31Ids = {}       
        self.sessions = {}


    def loadEntrants(self):
        '''Read entrants from JSON'''

        csvPath = os.path.join(self.path, CONFIG_DIR, ENTRANTS_CSV)
        
        with open(csvPath, 'r') as f:
            csvReader = csv.reader(f)
            headers = next(csvReader)

            for values in csvReader:
                entrant = Entrant(headers, values)
                if entrant.entrantId not in self.entrants:
                    self.entrants[entrant.entrantId] = entrant
                else:
                    raise ValueError('Duplicate entrant ID "{}"'.format(entrant.entrantId))

        if verbose:
            print('{} entrants loaded'.format(len(self.entrants)))


    def indexGt31s(self):
        '''Create GT-31 indices for entrants'''

        for entrantId in self.entrants.keys():
            entrant = self.entrants[entrantId]

            if 'GT31 ID' in entrant.entrantDict:
                self.gt31Ids[entrant.entrantDict['GT31 ID'].upper()] = entrant
        

    def summariseEntrants(self):
        '''Print summary of entrants'''
        
        pass


    def loadSessions(self):
        '''Load all of the event sessions - absence of config will result in an exception (intentional)'''
        
        dataPaths = sorted(glob.glob(os.path.join(self.path, GPSDATA_DIR, '20[0-9][0-9][0-1][0-9][0-3][0-9]')))
        
        for dataPath in dataPaths:
            configPath = os.path.join(self.path, SESSIONS_DIR, os.path.basename(dataPath))
            session = Session(self.entrants, dataPath, configPath)

            session.loadConfig()
            session.loadRuns(self.gt31Ids)
            
            # TODO - session.processRuns()

            self.sessions[session.date] = session
            

    def processEvent(self):
        '''Read entrants from config folder'''

        if verbose:
            print('Processing {}...'.format(self.year))

        self.loadEntrants()  
        #self.indexGt31s()
        #self.loadSessions()

        if verbose:
            print('All done!\n')

## Process Years

Process all available years

In [13]:
pc1 = time.perf_counter()

In [14]:
# Read main config
filename = os.path.join(projdir, CONFIG_DIR, CONFIG_JSON)
with open(filename, 'r', encoding='utf-8') as f:
    jsonTxt = f.read()
    config = json.loads(jsonTxt)

In [15]:
# Only process the current year (for now)
eventPaths = sorted(glob.glob(os.path.join(projdir, EVENTS_DIR, '20[0-9][0-9]')))
year = datetime.now().year
for eventPath in eventPaths:
    if int(os.path.basename(eventPath)) <= year:
        if int(os.path.basename(eventPath)) == year or wsw_full_refresh:
            event = Event(eventPath)
            event.processEvent()

Processing 2010...
125 entrants loaded
All done!

Processing 2011...
125 entrants loaded
All done!

Processing 2012...
130 entrants loaded
All done!

Processing 2013...
96 entrants loaded
All done!

Processing 2014...
88 entrants loaded
All done!

Processing 2015...
95 entrants loaded
All done!

Processing 2016...
90 entrants loaded
All done!

Processing 2017...
84 entrants loaded
All done!

Processing 2018...
87 entrants loaded
All done!

Processing 2019...
68 entrants loaded
All done!

Processing 2021...
50 entrants loaded
All done!



In [16]:
pc2 = time.perf_counter()
print("Reports completed in %0.2f seconds" % (pc2 - pc1))

Reports completed in 0.12 seconds


## All Done!